In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np

import time

In [2]:
def click_btn(clickable):
    for btn in clickable:
        try:
            btn.click()
        except:
            pass

In [3]:
# startpoint = 0
# count = 1000
# prefix_url = "https://growdiaries.com/explore"

service = Service('./chromedriver.exe')
op = webdriver.ChromeOptions()
op.add_argument('--headless')
# elements = []
# url_df = pd.DataFrame(data={"DiaryName": "", "Url": ""})

אחרי שניסיתי לטעון בדפדפן המון אופציות, השתמשתי בפוסטמן בשביל לבדוק את הפילטר של בקשת האיפיאי וראיתי שאני יכול למשוך קובץ של 2000 לינקים ליומנים
שמרתי את הדפי ווב של הקישורים ואז יצרתי דאטהבייס של קישורים ללינקים.

In [4]:
# driver = webdriver.Chrome(service=service, options=op)
# while startpoint < 10000:
#     query = f"?action=loadpage&category=all&tags=harvested&start={startpoint}&count={count}&undefined=&ajax=true&ust=9c824c5a63187dd5f579502e26ac3594c3dbb8752fd954432a4d6e1b0ff2e41f8654e9f3"
#     final_url = prefix_url + query
#     driver.get(final_url)
#     elements= driver.find_elements(By.XPATH, "//a[@class='name']")
#     startpoint += count
#     for element in elements:
#         temp_df = pd.DataFrame({"DiaryName": [element.text], "Url": [element.get_attribute('href')]})
#         url_df = pd.concat([url_df, temp_df], ignore_index=True)
# driver.quit()


In [5]:
# url_df.to_csv("Diary Links.csv", index=False)

In [6]:
url_df = pd.read_csv("Diary Links.csv")

In [17]:
url_df['Url'].loc[2095]

'https://growdiaries.com/diaries/154116-grow-journal-by-yan607'

כעת אני עובר על כל אחד מהקישורים ושומר את המידע בצ'אנקים של 1000 רשומות

In [73]:
driver = webdriver.Chrome(service=service, options=op)
count = 2000
const = 1000
while count < 10000:
    det_df = pd.DataFrame()
    for j, url in enumerate(url_df["Url"][count:count+const]):
        driver.get(url)
        driver.maximize_window()
        clickable = driver.find_elements(By.XPATH, "//div[@class='btn_less']")
        click_btn(clickable)
        report_items = driver.find_elements(By.XPATH, "//div[contains(@class,'report_items')]//*[@class='info']")
        outcome = driver.find_elements(By.XPATH, "//div[contains(@class,'parameters_item')]")
        likes = driver.find_element(By.XPATH, "//div[@class='report_statistic']//div[.//@class='icon-leaf-like']")
        # add comments as well!!!
        d = {"diary_name": url_df['DiaryName'].loc[count+j], "strain": "", "strains_company":"", "light_watt": "", "nutrients": "", "indoor": "", "watering": "", "soil": "", "germination": "", 'grow_techniques': "", "grow_room_size": "",
    "weeks_to_harvest": "", "num_of_plants": "",likes: likes.text, "bud_dry_weight": "", "bud_wet_weight": ""}

        for i, item in enumerate(report_items):
            try:
                text = item.text
                texts = text.split("\n")
                if i == 0:
                    d["strain"] = texts[0]
                    d['strains_company'] = texts[1]
                elif "LED" in text:
                    d['light_watt'] += text.replace("\n", " ") + ","
                elif "Nutrients" in text:
                    d['nutrients'] += texts[0] +","
                elif "Room Type" in text:
                    d['indoor'] = True
                elif "Watering" in text:
                    d['watering'] = texts[0]
                elif "Soil" in text or "Grow" in text:
                    d['soil']+= text.replace("\n", " ")+","
                elif "Germination" in text:
                    d['germination'] = texts[0]
                elif text.find("Week") > 0 and len(texts[0]) > 1:
                    d['grow_techniques'] += texts[0] + ","
                elif text.find("Tent") > 0:
                    d['grow_room_size'] = texts[0]
            except:
                pass
        for i, details in enumerate(outcome):
            try:
                text = details.text
                texts = text.split("\n")
                if i == 0:
                    d["weeks_to_harvest"] = texts[1]
                elif "BUD WET WEIGHT" in text:
                    d['bud_wet_weight']= texts[1]
                elif "BUD DRY WEIGHT" in text:
                    d['bud_dry_weight']= texts[1]
                elif "NUMBER OF PLANTS HARVESTED" in text:
                    d['num_of_plants'] = texts[1].split(" ")[0]
                elif "TOTAL LIGHT POWER" in text:
                    d['light_watt'] = texts[1]
                elif "GROW ROOM" in text:
                    d['grow_room_size']= texts[1]
            except:
                pass
        det_df = pd.concat([det_df,pd.DataFrame(d, index=[0])], ignore_index=True)
    det_df.to_csv(f"Data_{count}_{count+const}.csv", index=False)
    count += const

driver.quit()

In [41]:
driver = webdriver.Chrome(service=service, options=op)
det_df = pd.DataFrame()
d = []
for j, url in enumerate(url_df["Url"]):
    try:
        driver.get(url)
        driver.maximize_window()
        likes = driver.find_element(By.XPATH, "//div[@class='report_statistic']//div[.//@class='icon-leaf-like']")
        d.append(likes.text) 
    except:
        d.append(0)


In [ ]:
len(d)

208

In [ ]:
url_df['likes'] = d

'50'

In [ ]:
url_df.to_csv("Diary links with likes.csv", index=False)

In [40]:
url_df['Url'].loc[209]

'https://growdiaries.com/diaries/164797-grow-journal-by-piuswaxis'